In [26]:
import pandas as pd
import numpy as np

In [44]:
weather = pd.read_csv(
    "https://raw.githubusercontent.com/zonination/weather-us/master/nyc.csv",
    parse_dates=["Date"],
).drop("Unnamed: 0", axis=1)
s = weather['PrecipitationIn'].replace({'T':0}).astype(np.float32)
weather['PrecipitationIn'] = s.where(s < 5, 0)

nyc = pd.read_csv(
    "../NYCArrests.csv",
    usecols=[
        "ARREST_DATE",
        "PD_DESC",
        "OFNS_DESC",
        "ARREST_BORO",
        "AGE_GROUP",
        "PERP_SEX",
        "PERP_RACE",
        "Latitude",
        "Longitude",
    ],
    parse_dates=["ARREST_DATE"],
    cache_dates=True,
)
sp500 = pd.read_csv("../S&P500.csv", parse_dates=["Date"])

In [45]:
to_merge1 = sp500[["Date", "Close", 'Open', 'Volume']]
to_merge2 = (
    nyc["ARREST_DATE"]
    .value_counts()
    .reset_index(name="Num Arrests")
    .rename({"index": "Date"}, axis=1)
)
to_merge3 = weather[["Date", "Mean.TemperatureF", "PrecipitationIn", "Events", 'season']]

In [46]:
daily = to_merge1.merge(to_merge2).merge(to_merge3)

In [47]:
daily["Year"] = daily["Date"].dt.year
daily["Week"] = daily["Date"].dt.week
daily["Month"] = daily["Date"].dt.month

In [48]:
weekly = daily.groupby(["Year", "Week"]).mean().reset_index()


In [49]:
daily.to_csv('daily.csv', index = False)
weekly.to_csv('weekly.csv', index = False)

In [33]:
daily['Events'].str.split('-').explode().unique()

array(['Rain', nan, 'Fog', 'Snow', 'Thunderstorm'], dtype=object)

1 degree = 111 km

.1 degree = 11.1 km

.01 degree = 1.11 km

.001 degree = .111 km  = 111m

In [40]:
precision = 1000
nyc["RoundedLat"] = (nyc["Latitude"] * precision).round() / precision
nyc["RoundedLong"] = (nyc["Longitude"] * precision).round() / precision

In [41]:
to_graph = (
    nyc.groupby(["RoundedLat", "RoundedLong"]).size().reset_index(name="Count").dropna()
)
to_graph = to_graph[to_graph["RoundedLat"] < 41]
to_graph.columns

Index(['RoundedLat', 'RoundedLong', 'Count'], dtype='object')

In [45]:
"pk.eyJ1IjoiYXNkZmZhIiwiYSI6ImNqeTdwMHRjNDAxc3ozZHBob240ZWMwd3UifQ.7YZDqSZ_EZx7-78pnR0v_g"

'pk.eyJ1IjoiYXNkZmZhIiwiYSI6ImNqeTdwMHRjNDAxc3ozZHBob240ZWMwd3UifQ.7YZDqSZ_EZx7-78pnR0v_g'